<a href="https://colab.research.google.com/github/vusalamusayeva/pneumonia-detection/blob/main/Pneumonia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pneumonia detection

**Runtime type**

In [ ]:
import torch

# GPU
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Change runtime type to GPU")

print(f"PyTorch: {torch.__version__}")

GPU: Tesla T4
Memory: 15.8 GB
PyTorch: 2.9.0+cu126


**Libraries**

In [ ]:
!pip install -q pydicom opencv-python pandas numpy matplotlib seaborn scikit-learn pillow

# Import test
import pydicom
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
import os

print("All libraries are done")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.6 MB/s eta 0:00:00
All libraries are done


**for directory problem**

In [ ]:
import os
%cd /content

!rm -rf /content/yolov5

!git clone https://github.com/ultralytics/yolov5

if os.path.exists('/content/yolov5'):
    print(" Directory ")

    %cd /content/yolov5

    !pip install -q -r requirements.txt

    print("\n Yolov5 is done")

    %cd /content
else:
    print("Problem")

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17802, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 17802 (delta 7), reused 4 (delta 4), pack-reused 17788 (from 2)
Receiving objects: 100% (17802/17802), 16.97 MiB | 8.93 MiB/s, done.
Resolving deltas: 100% (12139/12139), done.
 Directory 
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 7.2 MB/s eta 0:00:00

 Yolov5 is done
/content


In [ ]:
if os.path.exists('/content/yolov5'):
    !rm -rf /content/yolov5

!git clone https://github.com/ultralytics/yolov5 --quiet
print(" YOLOv5 repository cloned")

%cd /content/yolov5

!pip install -q -r requirements.txt

print("\n YOLOv5 is done")

!python --version

 YOLOv5 repository cloned
/content/yolov5

 YOLOv5 is done
Python 3.12.12


In [ ]:
from google.colab import files

# Kaggle.json checking
if not os.path.exists('/root/.kaggle/kaggle.json'):
    print("kaggle.json file:")
    print("   (Kaggle.com → Account → Create New API Token)")
    uploaded = files.upload()

    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    print("\n Kaggle API was confugration")
else:
    print(" Kaggle API have done confugration")

!kaggle --version

kaggle.json file:
   (Kaggle.com → Account → Create New API Token)


Saving kaggle.json to kaggle.json

 Kaggle API was confugration
Kaggle API 1.7.4.5


In [ ]:
if os.path.exists('/content/rsna_data/stage_2_train_labels.csv'):
    print("Dataset exists")

    df = pd.read_csv('/content/rsna_data/stage_2_train_labels.csv')
    print(f" Train labels: {len(df)} rows")
    print(f" Unique patients: {df['patientId'].nunique()}")

else:
    print(" Dataset doesn't exist and is downloading")

    !kaggle competitions download -c rsna-pneumonia-detection-challenge

    if os.path.exists('rsna-pneumonia-detection-challenge.zip'):

        !mkdir -p /content/rsna_data
        !unzip -q rsna-pneumonia-detection-challenge.zip -d /content/rsna_data

        !rm rsna-pneumonia-detection-challenge.zip

    else:
        print("Zip file doesn't dowload")

print("\n Files:")
!ls -lh /content/rsna_data/ 2>/dev/null || echo " rsna_data folder don't"

if os.path.exists('/content/rsna_data/stage_2_train_labels.csv'):
    df = pd.read_csv('/content/rsna_data/stage_2_train_labels.csv')
    print(f"\n dataset is done ")
    print(f"   Labels: {len(df)} rows")
    print(f"   Patients: {df['patientId'].nunique()}")
else:
    print("\n csv doesn't find")

 Dataset doesn't exist and is downloading
100% 3.66G/3.66G [02:31<00:00, 95.5MB/s]
100% 3.66G/3.66G [02:31<00:00, 26.0MB/s]

 Files:
total 5.2M
-rw-r--r-- 1 root root   55 Dec 11  2019 'GCP Credits Request Link - RSNA.txt'
-rw-r--r-- 1 root root 1.6M Dec 11  2019  stage_2_detailed_class_info.csv
-rw-r--r-- 1 root root 156K Dec 11  2019  stage_2_sample_submission.csv
drwxr-xr-x 2 root root 216K Feb  4 12:24  stage_2_test_images
drwxr-xr-x 2 root root 1.9M Feb  4 12:25  stage_2_train_images
-rw-r--r-- 1 root root 1.5M Dec 11  2019  stage_2_train_labels.csv

 dataset is done 
   Labels: 30227 rows
   Patients: 26684


In [ ]:
if os.path.exists('/content/yolo_dataset'):
    !rm -rf /content/yolo_dataset

dataset_path = Path('/content/yolo_dataset')
dataset_path.mkdir(exist_ok=True)

for split in ['train', 'val']:
    (dataset_path / split / 'images').mkdir(parents=True, exist_ok=True)
    (dataset_path / split / 'labels').mkdir(parents=True, exist_ok=True)

!ls -R /content/yolo_dataset/

/content/yolo_dataset/:
train  val

/content/yolo_dataset/train:
images	labels

/content/yolo_dataset/train/images:

/content/yolo_dataset/train/labels:

/content/yolo_dataset/val:
images	labels

/content/yolo_dataset/val/images:

/content/yolo_dataset/val/labels:


In [ ]:
def convert_dicom_to_png(patient_id, dicom_dir, output_dir):

    try:
        dcm_path = os.path.join(dicom_dir, f"{patient_id}.dcm")
        dcm = pydicom.dcmread(dcm_path)
        image = dcm.pixel_array

        image = ((image - image.min()) / (image.max() - image.min()) * 255).astype(np.uint8)

        output_path = os.path.join(output_dir, f"{patient_id}.png")
        cv2.imwrite(output_path, image)

        return True, image.shape
    except Exception as e:
        return False, None

def create_yolo_annotation(patient_id, labels_df, img_height, img_width, output_dir):
    patient_boxes = labels_df[labels_df['patientId'] == patient_id]

    annotation_file = os.path.join(output_dir, f"{patient_id}.txt")

    with open(annotation_file, 'w') as f:
        for _, row in patient_boxes.iterrows():
            if row['Target'] == 1:
                x_center = (row['x'] + row['width'] / 2) / img_width
                y_center = (row['y'] + row['height'] / 2) / img_height
                width = row['width'] / img_width
                height = row['height'] / img_height

                f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

In [ ]:
train_labels = pd.read_csv('/content/rsna_data/stage_2_train_labels.csv')

print(f" {len(train_labels)} rows")
print(f"{train_labels['patientId'].nunique()}")

all_patient_ids = train_labels['patientId'].unique()

train_ids, val_ids = train_test_split(all_patient_ids, test_size=0.2, random_state=42)

train_sample = train_ids[:2000]
val_sample = val_ids[:400]

print(f" Train: {len(train_sample)} photo")
print(f" Val: {len(val_sample)} photo")
print(f" General: {len(train_sample) + len(val_sample)} photo")

print("\n split is done")

 30227 rows
26684
 Train: 2000 photo
 Val: 400 photo
 General: 2400 photo

 split is done


In [ ]:
from tqdm.notebook import tqdm

dicom_dir = '/content/rsna_data/stage_2_train_images'
dataset_path = Path('/content/yolo_dataset')

train_success = 0

for patient_id in tqdm(train_sample, desc="Train"):
    success, shape = convert_dicom_to_png(
        patient_id,
        dicom_dir,
        dataset_path / 'train' / 'images'
    )

    if success:
        create_yolo_annotation(
            patient_id,
            train_labels,
            shape[0],  # height
            shape[1],  # width
            dataset_path / 'train' / 'labels'
        )
        train_success += 1

print(f" Train: {train_success}/{len(train_sample)} photo is done")

val_success = 0

for patient_id in tqdm(val_sample, desc="Val"):
    success, shape = convert_dicom_to_png(
        patient_id,
        dicom_dir,
        dataset_path / 'val' / 'images'
    )

    if success:
        create_yolo_annotation(
            patient_id,
            train_labels,
            shape[0],
            shape[1],
            dataset_path / 'val' / 'labels'
        )
        val_success += 1

print(f" Val: {val_success}/{len(val_sample)} photo is done")

train_imgs = len(os.listdir('/content/yolo_dataset/train/images'))
val_imgs = len(os.listdir('/content/yolo_dataset/val/images'))

print(f" Train images: {train_imgs}")
print(f" Val images: {val_imgs}")

Train:   0%|          | 0/2000 [00:00<?, ?it/s]

 Train: 2000/2000 photo is done


Val:   0%|          | 0/400 [00:00<?, ?it/s]

 Val: 400/400 photo is done
 Train images: 2000
 Val images: 400


In [ ]:
yaml_content = """# RSNA Pneumonia Detection Dataset Configuration

path: /content/yolo_dataset
train: train/images
val: val/images

# Classes
nc: 1
names: ['pneumonia']
"""

yaml_path = '/content/yolo_dataset/dataset.yaml'

with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(" dataset.yaml done")
print(f" Location: {yaml_path}")

print("\n content:")
print(yaml_content)

if os.path.exists(yaml_path):
    print(" file is done")
else:
    print("problem")

 dataset.yaml done
 Location: /content/yolo_dataset/dataset.yaml

 content:
# RSNA Pneumonia Detection Dataset Configuration

path: /content/yolo_dataset
train: train/images
val: val/images

# Classes
nc: 1
names: ['pneumonia']

 file is done


In [ ]:
%cd /content/yolov5

!python train.py \
    --img 640 \
    --batch 16 \
    --epochs 50 \
    --data /content/yolo_dataset/dataset.yaml \
    --weights yolov5m.pt \
    --cache \
    --device 0 \
    --project /content/pneumonia_model \
    --name final_model \
    --patience 15 \
    --save-period 10 \
    --exist-ok

/content/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2026-02-04 12:27:21.665207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770208041.879682    9344 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770208041.935133    9344 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for 

In [ ]:
from IPython.display import Image, display
import os

results_dir = '/content/pneumonia_model/final_model'

if os.path.exists(f'{results_dir}/results.png'):
    print("\n Training Metrics:")
    display(Image(filename=f'{results_dir}/results.png', width=1200))
else:
    print(" results.png didn't find")

if os.path.exists(f'{results_dir}/confusion_matrix.png'):
    print("\n Confusion date")
    display(Image(filename=f'{results_dir}/confusion_matrix.png', width=700))

if os.path.exists(f'{results_dir}/F1_curve.png'):
    print("\n F1 Curve:")
    display(Image(filename=f'{results_dir}/F1_curve.png', width=700))

if os.path.exists(f'{results_dir}/PR_curve.png'):
    print("\n Precision-Recall Curve:")
    display(Image(filename=f'{results_dir}/PR_curve.png', width=700))

if os.path.exists(f'{results_dir}/val_batch0_pred.jpg'):
    print("\n Validation Predictions:")
    display(Image(filename=f'{results_dir}/val_batch0_pred.jpg', width=1000))

weights_path = f'{results_dir}/weights/best.pt'
if os.path.exists(weights_path):
    file_size = os.path.getsize(weights_path) / (1024*1024)
    print(f"\n model is done")
    print(f" Path: {weights_path}")
    print(f" Size: {file_size:.2f} MB")
    print("\n model's training is done")
else:
    print("\n best.pt didn't find")

 results.png didn't find

 best.pt didn't find


In [ ]:
from pathlib import Path

%cd /content/yolov5

!python detect.py \
    --weights /content/pneumonia_model/final_model/weights/best.pt \
    --source /content/yolo_dataset/val/images \
    --img 640 \
    --conf 0.25 \
    --save-txt \
    --save-conf \
    --project /content/batch_results \
    --name final_validation \
    --exist-ok

label_dir = Path('/content/batch_results/final_validation/labels')

if label_dir.exists():
    total_images = 0
    pneumonia_detected = 0
    total_detections = 0

    for lf in label_dir.glob('*.txt'):
        total_images += 1
        if lf.stat().st_size > 0:
            pneumonia_detected += 1
            with open(lf, 'r') as f:
                total_detections += len(f.readlines())

    print(f"general train photo {total_images}")
    print(f"photo with pneumonia {pneumonia_detected}")
    print(f"normal {total_images - pneumonia_detected}")

    if total_images > 0:
        print(f"pneumonia percent {(pneumonia_detected/total_images*100):.1f}%")
        print(f"general detection {total_detections}")

[Errno 2] No such file or directory: '/content/yolov5'
/content
python3: can't open file '/content/detect.py': [Errno 2] No such file or directory


In [ ]:
from IPython.display import Image, display
from pathlib import Path


result_dir = Path('/content/batch_results/final_validation')
result_images = sorted(list(result_dir.glob('*.png')) + list(result_dir.glob('*.jpg')))

print(f"general photo count {len(result_images)}")

for i, img_path in enumerate(result_images, 1):
    print(f"\n{'='*60}")
    print(f"📸 Şəkil #{i}/{len(result_images)}: {img_path.name[:50]}")
    print(f"{'='*60}")

    display(Image(filename=str(img_path), width=700))

    label_file = result_dir / 'labels' / f"{img_path.stem}.txt"
    if label_file.exists() and label_file.stat().st_size > 0:
        with open(label_file, 'r') as f:
            lines = f.readlines()
        print(f" {len(lines)} PNEVMONIYA DETECTION")
        for j, line in enumerate(lines, 1):
            parts = line.strip().split()
            conf = float(parts[5]) if len(parts) > 5 else 0
            print(f"   Detection #{j}: confidence = {conf:.3f}")
    else:
        print("normal")

general photo count 0


In [ ]:
!ls /content


project.zip  sample_data
